# AI Stock Forecaster  
**(FMP + Kronos + FinText-TSFM | Signal-Only, Point-in-Time Safe)**

---

## 0) Project Explanation & Philosophy

### What this project is

This project builds a **decision-support forecasting model** that answers one core question:

> **Which AI stocks are most attractive to buy today, on a risk-adjusted basis, over the next 20 / 60 / 90 trading days?**

The system outputs **ranked stock recommendations and return distributions**, not trades.  
Its purpose is to generate **credible alpha signals** that survive realistic financial constraints.

The design explicitly accounts for:
- non-stationary market behavior,
- weak and noisy financial signals,
- transaction costs and liquidity effects,
- and strict point-in-time (PIT) correctness.

---

### What this project is NOT

This project does **not**:
- place trades,
- connect to brokers,
- optimize execution,
- or manage live capital.

Any portfolio-related logic exists **only to validate signal realism**, not to implement trading.

### Core modeling philosophy

1. **Ranking beats regression**  
   Relative ordering of stocks is more stable and economically useful than exact price prediction.

2. **Point-in-time correctness is non-negotiable**  
   Any signal unavailable at time *T* must not influence predictions at time *T*.

3. **Economic validity > statistical fit**  
   Signals must survive transaction costs, turnover, and regime shifts.

4. **Multiple weak signals > single strong model**  
   Combine complementary views:
   - price dynamics (Kronos),
   - return structure (FinText-TSFM),
   - fundamentals and context (tabular models).

## 1) System Outputs (Signal-Only)

At each rebalance date **T**, for each stock and horizon (20 / 60 / 90 trading days):

### Per-stock outputs
- **Expected excess return** vs benchmark (QQQ default; XLK/SMH optional)
- **Return distribution** (5th / 50th / 95th percentiles)
- **Alpha ranking score** (cross-sectional)
- **Confidence score** (calibrated uncertainty)
- **Key drivers** (feature blocks influencing the rank)

### Cross-sectional outputs
- Ranked list: **Top buys / neutral / avoid**
- Optional confidence buckets (high vs low confidence)

## 2) Scope & Validation Philosophy (Signal-Only)

### Scope
- The system produces **signals**, not trades.
- No execution or order placement logic is implemented.

### Why portfolio concepts still appear
Portfolio concepts (turnover, costs, constraints) are used **only for evaluation realism**, to answer:
> *Would these signals remain economically meaningful if followed by an investor?*

### Optional realism check
- Paper trading (e.g., Alpaca paper) may be used **post-hoc** to validate:
  - timestamp integrity,
  - universe construction,
  - signal stability.
- Paper trading results are **never** used for training or model selection.

## 3) Data & Point-in-Time Infrastructure (FMP-First)

### 3.1 Data sources
- **Market**: Daily OHLCV, splits, dividends
- **Fundamentals**: Income, balance sheet, cash flow (quarterly)
- **Metadata**: Sector, industry, shares outstanding, market cap
- **Events**: Earnings dates with announcement time
- **Benchmarks**: QQQ (default), optional XLK / SMH
- **Regime proxies**: VIX, market breadth, rate proxies

---

### 3.2 Point-in-time (PIT) rules

Each datapoint stores:
- `value`
- `observed_at` (first public release timestamp)
- `effective_from`
- `source`

Rules:
- Fundamentals are **as-reported**, never restated historically
- Forward-fill allowed **only after** `observed_at`
- No feature may use information released after the cutoff time

---

### 3.3 Daily cutoff policy (anti-lookahead)

- Fixed cutoff time (e.g., 4:00pm ET)
- Features for date *T* may only use data with timestamps ≤ cutoff(T)
- Earnings handling distinguishes pre-market vs after-close announcements

---

### 3.4 Data audits & bias detection

Automated checks:
- PIT violation scanner
- Survivorship reconstruction audit
- Corporate action sanity checks
- Missingness and outlier detection

**Success criteria**
- < 0.1% PIT violations
- Universe reproducible for any historical date
- All datasets auditable and replayable

## 4) Survivorship-Safe Dynamic Universe

### 4.1 Universe construction (critical)

At each rebalance date **T**:
- Start with all U.S. equities meeting liquidity and price thresholds
- Filter by AI-relevant sector / industry tags
- Select **Top N by market cap as-of T**
- Persist constituents with timestamp

Hardcoded "today's winners" are explicitly disallowed.

---

### 4.2 Delistings & mergers
- Delisted stocks remain in historical universes where data exists
- Missing data is explicitly modeled rather than silently dropped

**Success criteria**
- Constituents vary meaningfully through time
- Backtests include both winners and failures

## 5) Feature Engineering (Bias-Safe)

### 5.0 Readiness Checklist & Implementation Plan

#### Infrastructure Available (from Chapters 3-4) ✅
| Component | Module | What It Provides |
|-----------|--------|------------------|
| Prices | `FMPClient.get_historical_prices()` | Split-adjusted OHLCV with `observed_at` |
| Fundamentals | `FMPClient.get_income_statement()` etc. | With `fillingDate` for PIT |
| Volume/ADV | `DuckDBPITStore.get_avg_volume()` | Computed from OHLCV |
| Events | `EventStore` | EARNINGS, FILING, SENTIMENT with PIT |
| Earnings | `AlphaVantageClient` + `ExpectationsClient` | BMO/AMC timing, surprises |
| Regime/VIX | `FMPClient.get_index_historical()` | SPY, VIX for regime detection |
| Universe | `UniverseBuilder` | FULL survivorship via Polygon |
| ID Mapping | `SecurityMaster` | Stable IDs, ticker changes |
| Calendar | `TradingCalendarImpl` | NYSE holidays, cutoffs |
| Caching | All clients | `data/cache/*` directories |

#### API Keys Available ✅
- `FMP_KEYS` - Prices, fundamentals, profiles (free tier: 250/day)
- `POLYGON_KEYS` - Symbol master, universe (free tier: 5/min)
- `ALPHAVANTAGE_KEYS` - Earnings calendar (free tier: 25/day)

---

#### Chapter 5 TODO List ✅ COMPLETE

**5.1 Targets (Labels)** ✅
- [x] Implement forward excess return calculation vs QQQ benchmark
- [x] Create label generator for 20/60/90 trading day horizons
- [x] Ensure labels are strictly PIT-safe (no future leakage)
- [x] **v2: Total return labels (dividends included) - DEFAULT**

**5.2 Price & Volume Features** ✅
- [x] Momentum features (1m, 3m, 6m, 12m returns)
- [x] Volatility (realized vol, vol-of-vol)
- [x] Drawdown (max drawdown, current vs high)
- [x] Relative strength vs universe median
- [x] Beta vs benchmark (rolling window)
- [x] ADV and volatility-adjusted ADV

**5.3 Fundamental Features (Relative)** ✅
- [x] P/E vs own 3-year history (z-score)
- [x] P/S vs sector median
- [x] Margins vs sector peers
- [x] Revenue/earnings growth vs sector
- [x] All ratios rank-transformed cross-sectionally

**5.4 Event & Calendar Features** ✅
- [x] Days to next earnings
- [x] Days since last earnings
- [x] Post-earnings drift window indicator (PEAD 63 days)
- [x] Surprise magnitude (last N quarters)
- [x] Surprise streak and cross-sectional z-score
- [x] Filing recency (days since last 10-Q/10-K)

**5.5 Regime & Macro Features** ✅
- [x] VIX level and percentile (2-year window)
- [x] VIX regime classification (low/normal/elevated/high)
- [x] Market trend regime (bull/bear/neutral)
- [x] Sector rotation indicators (tech vs defensives)
- [x] All features timestamped with cutoff enforcement

**5.6 Missingness Masks** ✅
- [x] Create explicit "known at time T" indicators
- [x] Missingness as first-class feature (not just imputation)
- [x] Track data coverage statistics by category
- [x] Generate coverage reports

**5.7 Feature Hygiene & Redundancy** ✅
- [x] Cross-sectional z-score/rank standardization
- [x] Rolling Spearman correlation matrix
- [x] Feature clustering (identify blocks)
- [x] VIF diagnostics (tabular features)
- [x] Rolling IC stability checks
- [x] Sign consistency analysis

**5.8 Feature Neutralization (Diagnostics)** ✅
- [x] Sector-neutral IC computation
- [x] Beta-neutral IC computation
- [x] Sector+Beta neutral IC computation
- [x] Delta (Δ) reporting for interpretation

**Testing & Validation** ✅
- [x] Unit tests for each feature block (5.1-5.7 all have tests)
- [x] PIT violation scanner on all features
- [x] Univariate IC ≥ 0.03 check for strong signals (IC tools available)
- [x] IC stability across rolling windows (FeatureHygiene.compute_ic_stability)
- [x] Feature coverage > 95% (MissingnessTracker.compute_coverage_stats)

---

### 5.1 Targets (Labels) ✅ COMPLETE
**Implemented in `src/features/labels.py`**

**v2 (DEFAULT): Total Return Labels**
- Forward excess returns include dividends
- Formula: `TR_i,T(H) - TR_b,T(H)` where TR = price return + dividend yield
- Horizons: 20 / 60 / 90 trading days
- PIT-safe with `label_matured_at` timestamps

**v1 (Legacy): Price-Only Labels**
- Available via `label_version='v1'` flag
- For backward compatibility only

---

### 5.2 Price & volume features ✅ COMPLETE
**Implemented in `src/features/price_features.py`**

| Feature | Description |
|---------|-------------|
| `mom_1m/3m/6m/12m` | Returns over 21/63/126/252 trading days |
| `vol_20d/60d` | Annualized volatility |
| `vol_of_vol` | Volatility of rolling volatility |
| `max_drawdown_60d` | Maximum drawdown |
| `rel_strength_1m/3m` | Z-score vs universe |
| `beta_252d` | Beta vs QQQ benchmark |
| `adv_20d/60d` | Average daily dollar volume |

---

### 5.3 Fundamentals (relative, normalized) ✅ COMPLETE
**Implemented in `src/features/fundamental_features.py`**

Raw ratios are avoided — all features are RELATIVE:
- `pe_zscore_3y`: P/E vs own 3-year history
- `pe_vs_sector`: P/E relative to sector median
- `ps_vs_sector`: P/S relative to sector median
- `gross_margin_vs_sector`: Margins vs sector
- `revenue_growth_vs_sector`: Growth vs sector peers
- `roe_zscore`, `roa_zscore`: Quality metrics z-scored

---

### Time-Decay Sample Weighting (Training Policy) ✅
**Implemented in `src/features/time_decay.py`**

**Why time decay matters for AI stocks:**
- AI business models and the "AI regime" (2020+) differ from earlier eras
- Market microstructure evolves (HFT, retail flow)
- Many AI stocks didn't exist 15+ years ago — that's OK
- Recent observations are more relevant for forward predictions

**Recommended half-lives:**
| Horizon | Half-Life | Weight at 6y | Weight at 9y |
|---------|-----------|--------------|--------------|
| 20d     | 2.5 years | ~18%         | ~7%          |
| 60d     | 3.5 years | ~30%         | ~14%         |
| 90d     | 4.5 years | ~38%         | ~21%         |

---

### 5.4 Events & calendars ✅ COMPLETE
**Implemented in `src/features/event_features.py`**

| Feature | Description |
|---------|-------------|
| `days_to_earnings` | Days until next expected earnings |
| `days_since_earnings` | Days since last report |
| `in_pead_window` | Post-earnings drift window (63 days) |
| `last_surprise_pct` | Most recent surprise % |
| `avg_surprise_4q` | Rolling 4Q average |
| `surprise_streak` | Consecutive beats/misses |
| `surprise_zscore` | Cross-sectional z-score |
| `days_since_10k/10q` | Filing recency |
| `reports_bmo` | Typical announcement timing |

---

### 5.5 Regime & macro ✅ COMPLETE
**Implemented in `src/features/regime_features.py`**

| Feature | Description |
|---------|-------------|
| `vix_level`, `vix_percentile` | VIX level and 2-year percentile |
| `vix_regime` | low/normal/elevated/high |
| `market_return_5d/21d/63d` | SPY returns at various windows |
| `market_regime` | bull/bear/neutral (MA-based) |
| `above_ma_50`, `above_ma_200` | Price vs moving averages |
| `tech_vs_staples/utilities` | Sector rotation signals |

---

### 5.6 Availability & missingness masks ✅ COMPLETE
**Implemented in `src/features/missingness.py`**

| Feature | Description |
|---------|-------------|
| `coverage_pct` | Overall feature coverage (0-1) |
| `{category}_coverage` | Per-category availability |
| `has_{type}_data` | Boolean availability flags |
| `is_new_stock` | < 1 year of history |

**Key Philosophy:** Missingness is a SIGNAL, not just noise.

---

### 5.7 Feature Hygiene & Redundancy Control ✅ COMPLETE
**Implemented in `src/features/hygiene.py`**

| Component | Description |
|-----------|-------------|
| Cross-sectional standardization | z-score or rank-transform within date |
| Rolling Spearman correlation | Correlation matrix computation |
| Feature clustering | Hierarchical clustering to identify blocks |
| VIF diagnostics | Variance Inflation Factor (diagnostic, not filter) |
| IC stability analysis | Rolling IC with sign consistency tracking |

> **Principle**: A feature with IC 0.04 once and −0.01 later is worse than IC 0.02 stable forever.

---

### 5.8 Feature Neutralization ✅ COMPLETE
**Implemented in `src/features/neutralization.py`**

**Purpose:** For diagnostics ONLY (not training). Reveals WHERE alpha comes from.

| Component | Description |
|-----------|-------------|
| Sector-neutral IC | IC after removing sector effects |
| Beta-neutral IC | IC after removing market beta |
| Sector+Beta neutral IC | IC after removing both factors |
| Delta (Δ) reporting | neutral_IC - raw_IC for interpretation |

**Interpretation:**
- Large negative Δ_sector → feature was mostly sector rotation
- Large negative Δ_beta → feature was mostly market exposure
- Small Δ → alpha is genuinely stock-specific

---

**Feature success criteria** ✅
- > 95% completeness (post-masking)
- Strong univariate signals show IC ≳ 0.03
- No feature introduces PIT violations
- **Stability**: IC sign consistent across ≥70% of rolling windows
- **Redundancy understood**: Feature blocks documented, correlation matrix computed

## 6) Evaluation Framework (Core Credibility Layer) ✅ CLOSED & FROZEN

---

### 🔒 CHAPTER 6 FREEZE STATUS

**Status:** CLOSED & FROZEN (December 30, 2025)  
**Tests:** 413/413 passing  
**Commits:**
- `18bad8a` - Chapter 6: Closure fixes + freeze REAL baseline reference
- `7e6fa3a` - Chapter 6: Freeze REAL baseline reference artifacts

**Frozen Baseline Floor (REAL DuckDB Data):**

| Horizon | Best Baseline | Median RankIC | Quintile Spread | Hit Rate @10 | N Folds |
|---------|---------------|---------------|-----------------|--------------|---------|
| **20d** | `mom_12m_monthly` | **0.0283** | 0.0035 | 0.50 | 109 |
| **60d** | `momentum_composite_monthly` | **0.0392** | 0.0370 | 0.60 | 109 |
| **90d** | `momentum_composite_monthly` | **0.0169** | 0.0374 | 0.60 | 109 |

**Sanity Check:** ✅ PASSED (`naive_random` RankIC ≈ 0 for all horizons)

**Frozen Artifacts (tracked in git):**
- `evaluation_outputs/chapter6_closure_real/` - Baseline reference (IMMUTABLE)
- `BASELINE_FLOOR.json` - Metrics to beat for Chapter 7+
- `BASELINE_REFERENCE.md` - Usage instructions
- `CLOSURE_MANIFEST.json` - Commit hash (`bf2cf8e`), data hash (`5723d4c88b8ecba1...`)

**Data Snapshot:**
- Source: DuckDB (`data/features.duckdb`)
- Rows: 192,307 (2016-01-04 → 2025-02-19)
- Tickers: ~100 (AI universe)
- Horizons: 20d, 60d, 90d (TRADING DAYS)
- Label Version: v2 (total return with dividends)

**Reference Doc:** See `CHAPTER_6_FREEZE.md` for complete details.

**What This Means:**
- ✅ Chapter 6 evaluation pipeline is COMPLETE and may not be modified
- ✅ Baseline reference is FROZEN and is the immutable comparison anchor for Chapter 7+
- ✅ All future models must use this frozen pipeline and beat the frozen baseline floor
- ⚠️ Any changes to evaluation definitions require a new version and complete re-freeze

---

> **CRITICAL PHILOSOPHY**: "You've crossed the line where bad evaluation can ruin a good system."
>
> **Approach**: Be conservative. Let results look "boring" if they are. Resist the urge to tweak features/models early. If signals survive Chapter 6 as-is, Chapters 7-11 will feel almost easy.

### 6.0 Prerequisites Check ✅
- **Labels**: v2 total return (dividends), mature-aware, PIT-safe ✅
- **Features**: 5.1-5.8 complete, stable, interpretable, auditable ✅
- **Missingness**: Explicit, not dropped ✅
- **Regime**: Visible but not leaked ✅
- **Alpha attribution**: Neutralization working ✅
- **PIT discipline**: Scanner enforced, 0 CRITICAL violations ✅

---

### 6.0.2 Definition Lock ✅ IMPLEMENTED

> **CRITICAL**: All time conventions are locked in `src/evaluation/definitions.py`

**Canonical Definitions (FROZEN):**

| Parameter | Value | Unit | Enforcement |
|-----------|-------|------|-------------|
| **Horizons** | 20, 60, 90 | TRADING DAYS | `validate_horizon()` |
| **Embargo** | 90 | TRADING DAYS | `validate_embargo()` |
| **Rebalance** | 1st of month | Calendar day | Walk-forward splitter |
| **Pricing** | Close-to-close | - | Label generator |
| **Maturity** | label_matured_at <= cutoff_utc | UTC datetime | fold.filter_labels() |

**Anti-Leakage Enforcement (HARD CONSTRAINTS):**
```python
# Embargo validation (raises ValueError if < 90 TRADING DAYS)
from src.evaluation import validate_embargo
validate_embargo(90)  # ✅ Passes
validate_embargo(30)  # ❌ ValueError: "must be at least 90 TRADING DAYS"

# Maturity check (UTC datetime, not naive date)
from src.evaluation import get_market_close_utc, is_label_mature
cutoff_utc = get_market_close_utc(date(2023, 6, 15))  # 4 PM ET → UTC
is_label_mature(label_matured_at, cutoff_date)  # Rejects naive datetimes
```

**Purging Rules (Per-Row-Per-Horizon):**
- Train labels: Purge if T + H (trading days) > train_end
- Val labels: Purge if T - H (trading days) < train_end
- NOT a global rule that "happens to work because embargo = 90"

**End-of-Sample Eligibility:**
- All horizons (20/60/90) must be valid for an as-of date
- No partial horizons near end of evaluation period
- `require_all_horizons=True` in splitter

**Test Coverage:** 65/65 tests passing
- `tests/test_definitions.py`: 40 tests
- `tests/test_walk_forward.py`: 25 tests

---

### 6.0.1 Pre-Implementation Sanity Checks (COMPLETE BEFORE CODING)

> **CRITICAL**: These are not blockers—they are sanity locks. Complete both before writing any Chapter 6 code.

#### ✅ Sanity Check 1: Manual IC vs Qlib IC Parity Test

**Purpose:** Ensure adapter/indexing is correct before generating hundreds of evaluation reports.

**Test Protocol:**
```python
# One fold, one horizon, same predictions
fold = "2023-Q1"
horizon = 20

# Manual RankIC calculation
manual_rankic = df.groupby("date").apply(
    lambda x: spearmanr(x["prediction"], x["label"])[0]
).median()

# Qlib RankIC calculation
qlib_df = adapter.to_qlib_format(predictions, labels)
qlib_rankic = qlib.evaluate(qlib_df)["IC"].median()

# STOP if they don't match
assert abs(manual_rankic - qlib_rankic) < 0.001, "IC mismatch - fix adapter!"
```

**Acceptance:** Manual and Qlib RankIC must agree to 3 decimal places.

**If they don't match → STOP immediately:**
- Check MultiIndex formatting (datetime, instrument)
- Check date alignment (T vs T+H)
- Check for missing data handling differences
- Check for sign flips (prediction vs label)

#### ✅ Sanity Check 2: Experiment Naming Convention

**Purpose:** Prevent chaos when Recorder usage explodes across hundreds of experiments.

**Convention (LOCK THIS IN NOW):**
```
exp = ai_forecaster/
      horizon={20,60,90}/
      model={kronos_v0, fintext_v0, tabular_lgb, baseline_mom12m}/
      labels={v1_priceonly, v2_totalreturn}/
      fold={01, 02, ..., 40}/
```

**Example paths:**
```
ai_forecaster/horizon=20/model=kronos_v0/labels=v2/fold=03
ai_forecaster/horizon=60/model=baseline_mom12m/labels=v2/fold=12
ai_forecaster/horizon=90/model=tabular_lgb/labels=v2/fold=25
```

---

### 6.1 Walk-Forward Engine

**Expanding window** (not rolling):
- Grows forward, never shrinks
- Preserves long-term signal stability
- Avoids artificial lookback dependency

**Rebalance Cadence (LOCKED):**

| Frequency | Purpose | Details |
|-----------|---------|---------|
| **Monthly (Primary)** | Main evaluation | First trading day of month, ~110 points (2016-2025) |
| **Quarterly (Secondary)** | Robustness check | Supplementary slice only |

**Evaluation Date Range (LOCKED):**
```python
EVAL_START = "2016-01-01"  # Earliest reliable fundamentals + universe snapshots
EVAL_END = "2025-06-30"    # Conservative: guarantees 90d label maturity
```

**Why these dates?**
- **2016-01-01**: FMP fundamentals reliable, universe coverage sufficient
- **2025-06-30**: All 90d labels mature (PIT-safe), includes 2023-25 AI rally
- **Result**: ~110 monthly points, multiple regimes (pre-COVID, COVID, drawdown, AI mania)

**Universe snapshots:**
- Uses `stable_id` snapshots from Chapter 4 (survivorship-safe)
- Respects `label_matured_at` timestamps (PIT-safe)
- No forward-looking information leakage

**Time-decay sample weighting** (training only):
- From `src/features/time_decay.py`
- Horizon-specific half-lives: 2.5y (20d), 3.5y (60d), 4.5y (90d)
- Per-date normalization for cross-sectional ranking

---

### 6.1.1 Baselines (Models to Beat)

**3 baselines to beat:**

| Baseline | Feature(s) | Purpose |
|----------|-----------|---------|
| **A: `mom_12m`** | 12-month momentum | Primary naive baseline (embarrassing if we can't beat) |
| **B: `momentum_composite`** | `(mom_1m + mom_3m + mom_6m + mom_12m) / 4` | Stronger but transparent |
| **C: `short_term_strength`** | `mom_1m` or `rel_strength_1m` | Diagnostic for horizon sensitivity |

**+ 1 sanity baseline (not a target):**
- **`naive_random`**: Deterministic random scores (sanity check: RankIC ≈ 0, confirms no systematic bias)

**Critical Guardrail:**
All baselines run through **identical pipeline:**
- Same universe snapshots (`stable_id`)
- Same missingness handling
- Same neutralization setting (raw or sector/beta-neutral)
- Same cost diagnostic treatment (6.4)
- Same purging/embargo
- Same walk-forward splits

**No baseline shopping:** Adding more baselines = temptation to cherry-pick weak ones.

---

### 6.2 Label Hygiene
**Enforce maturity rule:**
- `label_matured_at <= asof` strictly enforced
- No label is used before it matures

**Horizon-aware purging:**
- Remove overlapping labels across training/validation folds
- Prevents leakage from correlated label windows

**Embargo = max horizon:**
- Gap between train and validation = 90 trading days
- Conservative cushion for all horizons (20/60/90)

---

### 6.3 Metrics (Ranking-First)

**Primary Metric: RankIC**
- Spearman correlation of predicted ranks vs actual excess returns
- More stable than Pearson IC (robust to outliers)
- Directly measures ranking quality

**IC by Regime:**
- VIX low/high quartiles
- Bull/bear markets (SPY 200-day MA)
- Sector rotation periods

**Top-Bottom Quintile Spread:**
- Return of top 20% - return of bottom 20%
- Measures practical exploitability

**Top-K Definition (LOCKED):**

| Metric | Primary | Secondary | Target |
|--------|---------|-----------|--------|
| **Top-K size** | Top-10 | Top-20 | - |
| **Churn** | Jaccard or % retained | - | < 30% |
| **Hit Rate** | Excess return > 0 | - | > 55% |

**Churn formula:**
```python
churn = 1 - len(set(top_k_t) & set(top_k_t_minus_1)) / len(set(top_k_t) | set(top_k_t_minus_1))
```

**Hit Rate definition:**
```python
hit = (top_k_portfolio_return - benchmark_return) > 0
hit_rate = hits / total_rebalances
```

---

### 6.4 Cost Realism (Diagnostic)

**Base costs:**
- 20 bps round-trip (conservative for liquid largecaps)
- Higher for small/mid caps if needed

**ADV-scaled slippage:**
- Function of position size / average daily dollar volume
- Penalizes illiquid names

**Question: "Does alpha survive?"**
- This is diagnostic, NOT optimization
- If alpha vanishes post-cost → reject signal
- If survives → document slippage sensitivity

---

### 6.5 Stability Reports

**IC decay plots:**
- How does IC degrade over time within a fold?
- Stable signals show flat or slow decay
- Rapid decay → overfitting or regime shift

**Regime-conditional performance:**
- IC in VIX high vs low
- IC in bull vs bear
- IC across sectors

**Churn diagnostics:**
- Top-10 ranking turnover month-over-month
- High churn (>50%) → unstable, costly
- Target: <30% for exploitability

---

### 6.6 Qlib Integration (Shadow Evaluator)

**Philosophy:** Use Microsoft's Qlib as a "shadow evaluator" for standardized reporting, NOT as a replacement for our core infrastructure.

**What Qlib Does for Us:**

| Chapter 6 Component | Qlib Feature |
|---------------------|--------------|
| **6.3 Metrics** | Built-in IC/RankIC analysis, monthly IC, regime IC, autocorrelation plots |
| **6.5 Reporting** | Quintile analysis, cumulative returns, long-short distribution, drawdown |
| **6.4 Cost Realism** | Backtest engine with configurable transaction costs (second opinion) |
| **Experiment Tracking** | Recorder system for managing walk-forward folds and model variants |

**Integration Pattern (Narrow & Safe):**
```
Our System (source of truth) → predictions + labels → Qlib → evaluation reports
```

**Data Flow:**
1. Our pipeline generates: `(date, ticker, prediction, label, optional_group)`
2. Qlib receives this DataFrame (not raw data/features)
3. Qlib outputs: standardized factor evaluation + backtest summaries

**What Qlib Does NOT Replace:**
- ❌ Universe construction (we keep stable_id + survivorship)
- ❌ Feature engineering (we keep PIT discipline + 5.1-5.8)
- ❌ Label generation (we keep v2 total return)
- ❌ Data storage (we keep DuckDB PIT store)

**Implementation:**
```python
# Adapter layer
def our_predictions_to_qlib_format(predictions_df, labels_df):
    qlib_df = pd.merge(predictions_df, labels_df, on=["date", "ticker"])
    qlib_df = qlib_df.set_index(["datetime", "instrument"])  # Qlib format
    return qlib_df

# Generate reports
from qlib.contrib.evaluate import backtest_daily
reports = backtest_daily(prediction=qlib_df, ...)
```

**References:**
- Qlib GitHub: https://github.com/microsoft/qlib
- Qlib Docs: https://qlib.readthedocs.io/en/latest/
- Evaluation: https://qlib.readthedocs.io/en/latest/component/report.html

---

**Acceptance Criteria:**
- ✅ Median walk-forward RankIC > baseline by ≥ 0.02
- ✅ Net-of-cost improvement: % positive folds ≥ baseline + 10pp (relative gate; frozen floor: 5.8%-40.1%)
- ✅ Top-10 ranking churn < 30% month-over-month
- ✅ Performance degrades gracefully under regime shifts
- ✅ NO PIT violations (enforced by scanner)

**Guardrails:**
- ❌ NO new features mid-evaluation
- ❌ NO retraining models to "fix" bad folds
- ❌ NO cherry-picking good time periods
- ❌ NO optimizing to costs (diagnostic only)
- ❌ NO hiding negative results

**Success = Boring Results That Don't Break**
- Median IC of 0.03-0.05 is GOOD
- Stable across regimes is EXCELLENT
- Survives costs is SUFFICIENT

## 7) Baseline Models (Models to Beat) ✅ IMPLEMENTED

---

### 7.0 Baseline Philosophy

Baselines establish the **floor** that models must clear. They serve three purposes:

1. **Sanity check**: If a model can't beat momentum, something is fundamentally wrong
2. **Value demonstration**: ML must add measurable value over transparent alternatives  
3. **Stability anchor**: Frozen baselines prevent "drifting targets" during model iteration

**Critical rule**: Baselines are **locked before any model is trained**. No baseline shopping.

---

### 7.1 Baseline Categories

All baselines run through the **identical evaluation pipeline** as models:
- Same universe snapshots (stable_id + survivorship)
- Same walk-forward folds (purging/embargo/maturity)
- Same EvaluationRow contract
- Same metrics, costs, and stability reports

#### 7.1.1 Factor Baselines (Models Must Beat)

| Baseline | Description | Formula | Purpose |
|----------|-------------|---------|---------|
| `mom_12m` | 12-month momentum | `score = mom_12m` | **Primary naive baseline** — If models can't beat this, something is wrong |
| `momentum_composite` | Multi-horizon momentum | `score = (mom_1m + mom_3m + mom_6m + mom_12m) / 4` | **Stronger transparent baseline** — Realistic bar for "is ML worth it?" |
| `short_term_strength` | 1-month momentum | `score = mom_1m` | **Diagnostic baseline** — Exposes horizon sensitivity and mean-reversion regimes |

#### 7.1.2 Sanity Baselines (Pipeline Verification Only)

| Baseline | Description | Formula | Purpose |
|----------|-------------|---------|---------|
| `naive_random` | Deterministic random | `score = hash(as_of_date, horizon, stable_id)` | **Pipeline sanity** — If RankIC ≠ ~0, evaluation is hallucinating alpha |

**Note**: `naive_random` should NEVER be used as a "bar to clear". It's purely a sanity check.

#### 7.1.3 ML Baselines (Tuned, Then Frozen)

| Baseline | Description | Status |
|----------|-------------|--------|
| `tabular_lgb` | LightGBM on feature stack | 🔄 TODO (Chapter 7) |

The ML baseline establishes: "Does deep learning add value over tuned gradient boosting?"

---

### 7.2 Baseline Gates (Pass/Fail Thresholds)

These are the minimum thresholds for each baseline category:

| Gate | Metric | Threshold | What It Means |
|------|--------|-----------|---------------|
| **Factor Gate** | median RankIC (best factor) | ≥ 0.02 | Momentum signal exists in data |
| **ML Gate** | median RankIC (tabular_lgb) | ≥ 0.05 | ML extracts signal beyond factors |
| **Model Gate** | median RankIC (model) | ≥ ML baseline + 0.02 | Deep learning adds value |

**Gating policy**:
- Factor gate must pass before proceeding (confirms data quality)
- ML gate establishes the "ML floor" for TSFM models
- TSFM models must beat tuned ML baseline on **median OOS RankIC**

---

### 7.3 Running Baselines

```python
from pathlib import Path
from src.evaluation import (
    ExperimentSpec,
    run_experiment,
    SMOKE_MODE,
    FULL_MODE,
    FACTOR_BASELINES,
    SANITY_BASELINES,
)

# Run all factor baselines (SMOKE mode for CI)
for baseline_name in FACTOR_BASELINES:
    spec = ExperimentSpec.baseline(baseline_name, cadence="monthly")
    results = run_experiment(
        experiment_spec=spec,
        features_df=features,  # Your features DataFrame
        output_dir=Path("evaluation_outputs"),
        mode=SMOKE_MODE  # or FULL_MODE for production
    )
    print(f"{baseline_name}: {results['n_folds']} folds")

# Run sanity baseline (verify ~0 RankIC)
spec = ExperimentSpec.baseline("naive_random", cadence="monthly")
sanity_results = run_experiment(spec, features, Path("evaluation_outputs"), SMOKE_MODE)
assert abs(sanity_results["median_rankic"]) < 0.05, "Pipeline sanity failed!"
```

---

### 7.4 Acceptance Criteria (Models Must Clear)

| Criterion | Threshold | Rationale |
|-----------|-----------|-----------|
| **RankIC Lift** | Model median RankIC >= best baseline + 0.02 | ML must add meaningful signal |
| **Net-Positive Folds** | >= 70% of folds positive after base costs | Signal must survive realistic trading |
| **Top-10 Churn** | Median < 30% | Rankings must be stable enough to trade |
| **No Collapse** | 0 folds with negative median RankIC | Robust across regimes |

These criteria are computed via:
```python
from src.evaluation import compute_acceptance_verdict, save_acceptance_summary

verdict = compute_acceptance_verdict(
    model_summary,
    baseline_summaries={
        "mom_12m": ..., 
        "momentum_composite": ..., 
        "short_term_strength": ...,
        "tabular_lgb": ...  # Once implemented
    },
    cost_overlays=cost_df,
    churn_df=churn_df
)
save_acceptance_summary(verdict, Path("outputs"), "model_vs_baselines")
```

---

### 7.5 FULL_MODE Reference Run (Required Before Model Work)

**Before training any model**, freeze a FULL_MODE baseline run:
- **Range**: 2016-01-01 → 2025-06-30 (locked)
- **Cadence**: Monthly (primary), Quarterly (robustness)
- **Outputs**: All baselines, all 3 horizons (20/60/90)

This produces:
```
evaluation_outputs/
├── baseline_mom_12m_monthly/
│   ├── eval_rows.parquet
│   ├── per_date_metrics.csv
│   ├── fold_summaries.csv
│   ├── cost_overlays.csv
│   ├── stability_scorecard.csv
│   └── REPORT_SUMMARY.md
├── baseline_momentum_composite_monthly/
├── baseline_short_term_strength_monthly/
├── baseline_naive_random_monthly/
├── baseline_tabular_lgb_monthly/  # Once implemented
└── BASELINE_REFERENCE.md  # Frozen floor for all horizons
```

**Freeze requirements**:
- Git commit hash of the run
- Run configuration (cadence, horizons, eval range, cost scenarios)
- Data snapshot identity (DuckDB hash + row counts)
- Output directory + manifest
- Environment snapshot (Python version + pip freeze)

---

### 7.6 Tabular ML Baseline ✅ IMPLEMENTED + FROZEN

The ML baseline uses LightGBM Regressor on the same feature stack as factor baselines.

#### 7.6.1 Training Protocol (Implemented)

```python
# Per-fold training using walk-forward split
for fold in walk_forward_folds:
    # Uses same purging + embargo + maturity
    train_data = fold.train_df
    val_data = fold.val_data
    
    # Horizon-specific models (separate model per horizon)
    for horizon in [20, 60, 90]:
        model = lgb.LGBMRegressor(  # Regressor (not Ranker) for continuous returns
            objective="regression_l1",
            metric="rmse",
            n_estimators=100,  # FROZEN after initial tuning
            learning_rate=0.05,
            max_depth=5,
            num_leaves=31,
            random_state=42,
            # Time-decay weighting: recent data weighted higher (half-life=252d)
        )
        model.fit(
            train_data[features],
            train_data[f"excess_return_{horizon}d"],
            group=train_data.groupby("as_of_date").size().values,
            sample_weight=time_decay_weights  # Exponential decay
        )
```

#### 7.6.2 Frozen Baseline Floor (COMPLETE)

**Status:** FROZEN at tag `chapter7-tabular-lgb-freeze`  
**Artifacts:** `evaluation_outputs/chapter7_tabular_lgb_full/`  
**Reference:** `BASELINE_REFERENCE.md` + `CLOSURE_MANIFEST.json`

| Horizon | Frozen Factor Floor | tabular_lgb (ML) | Lift |
|---------|---------------------|------------------|------|
| 20d | 0.0283 | **0.1009** | **+0.0726** |
| 60d | 0.0392 | **0.1275** | **+0.0883** |
| 90d | 0.0169 | **0.1808** | **+0.1639** |

**Implementation complete:**
1. ✅ One-time param tuning completed
2. ✅ Params frozen and recorded
3. ✅ FULL_MODE reference run executed (109 monthly folds, 36 quarterly)
4. ✅ Artifacts frozen with git tag
5. ✅ **ML baseline is now immutable**

---

### 7.7 Output Schema (EvaluationRow Contract)

Every baseline (and model) produces rows in this exact format:

| Field | Type | Description |
|-------|------|-------------|
| `as_of_date` | date | Evaluation date |
| `ticker` | str | Stock symbol |
| `stable_id` | str | Survivorship-safe identifier |
| `horizon` | int | 20, 60, or 90 trading days |
| `fold_id` | str | Walk-forward fold |
| `score` | float | Ranking score (HIGHER = BETTER) |
| `excess_return` | float | v2 total return vs benchmark |
| `adv_20d` | float | Average daily volume (cost realism) |

**Rules**:
- No duplicates per (as_of_date, stable_id, horizon)
- Missing score/return → row dropped (logged)
- Tie-breaking: deterministic via stable_id

---

### 7.8 Implementation Location

```
src/evaluation/baselines.py     # Baseline definitions and scoring
src/evaluation/run_evaluation.py # End-to-end runner (SMOKE/FULL modes)
tests/test_baselines.py         # 39 tests (monotonicity, determinism, etc.)
tests/test_end_to_end_smoke.py  # 22 integration tests
```

**Tests**: 61 tests specific to baselines and runner, all passing.

---

### 7.9 What "Done" Looks Like

When Chapter 7 baseline work is complete:

```
evaluation_outputs/
├── baseline_mom_12m_monthly/
├── baseline_mom_12m_quarterly/
├── baseline_momentum_composite_monthly/
├── baseline_momentum_composite_quarterly/
├── baseline_short_term_strength_monthly/
├── baseline_short_term_strength_quarterly/
├── baseline_naive_random_monthly/  # Sanity check
├── baseline_tabular_lgb_monthly/   # ML baseline
├── baseline_tabular_lgb_quarterly/
└── BASELINE_REFERENCE.md
```

The `BASELINE_REFERENCE.md` contains:
- Median RankIC per horizon (20/60/90), monthly and quarterly
- Churn and cost-survival diagnostics
- Pass/fail for each baseline gate
- Frozen commit hash + output path

**This becomes the only reference point for Chapter 8+ (Kronos, FinText, Fusion).**

## 8) Kronos Module (Price Dynamics)

### 8.0 Prerequisites ⚠️ REQUIRED BEFORE STARTING

**Feature Store Expansion (Pre-Chapter-8 Task)**

Current DuckDB only has **7 features**. Need to expand to **50 features** before Kronos:

**Priority 1: Price/Volume (14 features) - REQUIRED**
- Add: `vol_of_vol`, `max_drawdown_60d`, `dist_from_high_60d`
- Add: `rel_strength_1m`, `rel_strength_3m`, `beta_252d`
- Status: Code exists in `src/features/price_features.py`, needs wiring to DuckDB

**Priority 2: Events/Earnings (12 features) - CRITICAL for Earnings Gap Issue**
- Add: `days_to_earnings`, `days_since_earnings`, `in_pead_window`, `pead_window_day`
- Add: `last_surprise_pct`, `avg_surprise_4q`, `surprise_streak`, `surprise_zscore`, `earnings_vol`
- Add: `days_since_10k`, `days_since_10q`, `reports_bmo`
- Status: Code exists in `src/features/event_features.py`, needs wiring to DuckDB

**Priority 3: Regime/Macro (15 features) - Optional for Ch8, Required for Ch12**
- Add: VIX features, market regime, sector rotation
- Status: Code exists in `src/features/regime_features.py`

**Priority 4: Fundamentals (7 features) - Optional (FMP Premium available)**
- Add: P/E, P/S, margins, ROE/ROA
- Status: Code exists in `src/features/fundamental_features.py`

**Action Required:**
```bash
# Expand scripts/build_features_duckdb.py to call all feature generators
# Then rebuild:
python scripts/build_features_duckdb.py --auto-normalize-splits
```

**Note:** Chapter 7 baseline is frozen (uses only 13 features). Expanding DuckDB does NOT invalidate frozen artifacts.

---

### 8.1 Kronos Model Integration

- Input: OHLCV sequences
- Rolling / ReVIN-style normalization
- Outputs: embeddings and horizon-aware signals
- Fine-tuning via walk-forward only

**Kronos success criteria**
- Zero-shot IC measured
- Fine-tuning improves IC by ≥ 0.01
- Stable behavior across price level shifts
- Must beat frozen ML baseline: 0.1009/0.1275/0.1808 + 0.02

## 9) FinText-TSFM Module (Return Structure)

- Input: historical excess returns
- Year-specific checkpoints to reduce pretraining leakage
- Outputs: return distributions and embeddings

**FinText success criteria**
- Adds independent signal (low correlation with Kronos)
- Improves fusion IC consistently across regimes

## 10) NLP Sentiment (Separate)

- Finance-specific NLP model (news / transcripts)
- Strict cutoff-time enforcement

Sentiment is optional and never required.

## 11) Fusion Model (Ranking-First)

- Gated fusion of:
  - Kronos embeddings
  - FinText-TSFM embeddings
  - Tabular context features

  ### Training
- **Time-decay sample weighting** (from `src/features/time_decay.py`)
- Per-date normalization for cross-sectional ranking loss

### Objectives
- Primary: pairwise / listwise ranking loss
- Secondary: distribution calibration loss
- No pure MSE price regression

### Ablation gates
- Feature blocks removed if unstable
- Fusion must beat best single model

---

## 12) Regime-Aware Ensembling

- Components: Fusion, ML baseline, simple factor
- Regime detector (volatility / trend)
- Smooth, regularized ensemble weights

**Success criteria**
- Ensemble improves median IC
- Reduces variance across regimes

## 13) Calibration & Confidence

- Quantile calibration
- Confidence stratification

**Success criteria**
- Quantile coverage error < 5%
- High-confidence bucket materially outperforms

## 14) Monitoring & Research Ops

- Prediction logging with timestamps
- Matured-label scoring
- Feature and performance drift detection

Alerts:
- RankIC decay
- Calibration breakdown
- Ranking instability

## 15) Outputs & Interfaces

- Ranked stock lists
- Per-stock explanation summaries
- Batch scoring interface
- Full traceability of inputs and decisions

## 16) Global Research Acceptance Criteria

A model is considered **valid** if:

- Median walk-forward RankIC exceeds baseline by ≥ 0.02
- Net-of-cost performance positive in ≥ 70% of folds
- Top-10 ranking churn < 30% month-over-month
- Performance degrades gracefully under regime shifts
- No PIT or survivorship violations detected
